In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import json
from pathlib import Path
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import torch
import os

%matplotlib inline






In [2]:
load_dotenv()

True

In [3]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")


In [4]:
# Set the tickers for both the bond and stock portion of the portfolio
tickers = ["SPY"]

# Set timeframe to 1Day
timeframe = "1Day"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2000-10-13", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2022-10-13", tz="America/New_York").isoformat()



In [5]:
# Use the Alpaca get_bars function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
SPY = alpaca.get_bars(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df


SPY

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410,SPY
2015-12-02 05:00:00+00:00,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055,SPY
2015-12-03 05:00:00+00:00,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936,SPY
2015-12-04 05:00:00+00:00,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631,SPY
2015-12-07 05:00:00+00:00,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128,SPY
...,...,...,...,...,...,...,...,...
2022-10-07 04:00:00+00:00,369.06,373.2933,360.9400,362.79,107239188,665841,365.065822,SPY
2022-10-10 04:00:00+00:00,364.00,364.2100,357.6700,360.02,75459157,593460,360.745547,SPY
2022-10-11 04:00:00+00:00,358.30,363.0300,355.7100,357.74,91448816,688092,358.717517,SPY


In [6]:

# Reorganize the DataFrame
# Separate ticker data
SPY=SPY.drop(columns=['symbol'])

SPY



,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2015-12-01 05:00:00+00:00,209.42,210.8200,209.1100,210.68,97858418,337780,209.921410
2015-12-02 05:00:00+00:00,210.60,211.0000,208.2300,208.54,108069059,367013,209.563055
2015-12-03 05:00:00+00:00,208.90,209.1500,204.7511,205.58,166224154,546768,206.878936
2015-12-04 05:00:00+00:00,206.10,209.9700,205.9300,209.66,192878747,556731,208.178631
2015-12-07 05:00:00+00:00,209.20,209.7295,207.2000,208.27,102027111,374705,208.276128
...,...,...,...,...,...,...,...
2022-10-07 04:00:00+00:00,369.06,373.2933,360.9400,362.79,107239188,665841,365.065822
2022-10-10 04:00:00+00:00,364.00,364.2100,357.6700,360.02,75459157,593460,360.745547
2022-10-11 04:00:00+00:00,358.30,363.0300,355.7100,357.74,91448816,688092,358.717517
